In [142]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [143]:
cd gdrive/MyDrive/AI\ Challenge\ 1\ -\ 2021


[Errno 2] No such file or directory: 'gdrive/MyDrive/AI Challenge 1 - 2021'
/content/gdrive/MyDrive/AI Challenge 1 - 2021


In [144]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bird.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bucket.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/clock.npy


--2021-12-15 13:46:40--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.215.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143287056 (137M) [application/octet-stream]
Saving to: ‘car.npy.10’

car.npy.10          100%[===================>] 136.65M  53.9MB/s    in 2.5s    

2021-12-15 13:46:43 (53.9 MB/s) - ‘car.npy.10’ saved [143287056/143287056]

--2021-12-15 13:46:43--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bird.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 64.233.170.128, 108.177.11.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104720528 (100M) [application/octet-stre

In [145]:
import urllib.request
!mkdir data
base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
root = 'data'
class_names = ['car', 'bird', 'bucket', 'clock']
for c in class_names:
    path = base+c+'.npy'
    print(path)
    urllib.request.urlretrieve(path, f'{root}/{c}.npy')

mkdir: cannot create directory ‘data’: File exists
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bird.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bucket.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/clock.npy


In [146]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

In [147]:
N_OBJS = 4
OBJS = {0: "car", 1: "bird", 2: "bucket", 3: "clock"}

# number of samples to take in each class
N = 5000

# some other constants
N_EPOCHS = 10

# data files with orders like OBJS
files = ["car.npy", "bird.npy", "bucket.npy", "clock.npy"]

In [149]:
def load(dir, reshaped, files):
    "Load .npy or .npz files from disk and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    "Takes a list or a list of lists and returns its normalized form"

    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    "Takes a list or a list of lists and returns its denormalized form"

    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [150]:
objs = load("data/", False, ['car.npy'])

In [151]:
visualize(objs[0][90])

In [152]:
objs = load("data/", True, files)


In [153]:
objs = map(normalize, objs)


In [154]:
labels = make_labels(4, N)


In [155]:
Y_train = np_utils.to_categorical(y_train, 4)
Y_test = np_utils.to_categorical(y_test, 4)


In [156]:
y_train.shape
x_train.shape

(32000, 28, 28)

In [157]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))



In [158]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=10)

print("Training complete")

print("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print ("Accuracy: ", ((score + 0.0) / len(preds)) * 100)


model.save("objects"+ ".h5")
print("Model saved")

Epoch 1/10
1000/1000 [==============================] - 56s 55ms/step - loss: 0.4913 - accuracy: 0.9057
Epoch 2/10
1000/1000 [==============================] - 55s 55ms/step - loss: 0.1697 - accuracy: 0.9482
Epoch 3/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1387 - accuracy: 0.9577
Epoch 4/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1265 - accuracy: 0.9623
Epoch 5/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1100 - accuracy: 0.9649
Epoch 6/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1003 - accuracy: 0.9673
Epoch 7/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.0885 - accuracy: 0.9729
Epoch 8/10
1000/1000 [==============================] - 54s 54ms/step - loss: 0.0877 - accuracy: 0.9726
Epoch 9/10
1000/1000 [==============================] - 53s 53ms/step - loss: 0.0810 - accuracy: 0.9746
Epoch 10/10
1000/1000 [==============================] - 53s 53m